In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

R = 1.097373e7  # Constante de Rydberg (1/m)

# Clasificación espectral
def clasificar_radiacion(lambda_nm):
    if lambda_nm < 400:
        return "Ultravioleta (UV)"
    elif lambda_nm <= 700:
        return "Visible"
    else:
        return "Infrarrojo (IR)"

# Modelos atómicos
def calcular_longitud_onda_bohr(n1, n2, Z):
    if n1 >= n2:
        return None
    lambda_m = 1 / (R * Z**2 * (1/n1**2 - 1/n2**2))
    return lambda_m * 1e9  # nm

def calcular_longitud_onda_sommerfeld(n1, n2, Z):
    if n1 >= n2:
        return None
    lambda_m = 1 / (R * Z**2 * (1/n1**2 - 1/n2**2)) * 1.05
    return lambda_m * 1e9  # nm

def calcular_longitud_onda_schrodinger(n1, n2, Z):
    if n1 >= n2:
        return None
    lambda_m = 1 / (R * Z * (1/n1**2 - 1/n2**2))
    return lambda_m * 1e9  # nm

def calcular_longitud_onda_rutherford(n1, n2, Z):
    if n1 >= n2:
        return None
    lambda_m = 1 / (R * Z * (1/n1**2 - 1/n2**2))
    return lambda_m * 1e9  # nm

def seleccionar_modelo(modelo, n1, n2, Z):
    if modelo == 'Bohr':
        return calcular_longitud_onda_bohr(n1, n2, Z)
    elif modelo == 'Sommerfeld':
        return calcular_longitud_onda_sommerfeld(n1, n2, Z)
    elif modelo == 'Schrödinger':
        return calcular_longitud_onda_schrodinger(n1, n2, Z)
    elif modelo == 'Rutherford':
        return calcular_longitud_onda_rutherford(n1, n2, Z)

# Color para el espectro visible
def longitud_onda_a_color(l):
    if l < 400 or l > 700:
        return 'black'
    if l < 450:
        return 'violet'
    elif l < 495:
        return 'blue'
    elif l < 570:
        return 'green'
    elif l < 590:
        return 'yellow'
    elif l < 620:
        return 'orange'
    else:
        return 'red'

# Mostrar espectro visible
def mostrar_espectro(lambda_nm):
    fig, ax = plt.subplots(figsize=(6, 1))
    gradient = np.linspace(400, 700, 300)
    for wl in gradient:
        ax.axvline(x=wl, color=longitud_onda_a_color(wl), linewidth=4)

    ax.set_xlim(380, 720)
    ax.set_xticks([400, 500, 600, 700])
    ax.set_yticks([])
    ax.set_title("Espectro Visible (400–700 nm)", fontsize=12, fontweight='bold')

    if 400 <= lambda_nm <= 700:
        ax.axvline(x=lambda_nm, color='white', linestyle='--', linewidth=2)
        ax.text(lambda_nm, 0.1, f"{lambda_nm:.0f} nm", color='white', ha='center', fontsize=9, fontweight='bold')
    plt.show()

# Graficar niveles y transición
def graficar_transicion(modelo, n1, n2, Z):
    niveles = np.arange(1, max(n1, n2)+2)
    fig, ax = plt.subplots(figsize=(8, 6))

    def escalar(E):
        return -1 / E  # Mejora visual: separa más niveles superiores

    niveles_energias = {n: -Z**2 / n**2 for n in niveles}
    niveles_escalados = {n: escalar(E) for n, E in niveles_energias.items()}

    for n in niveles:
        y = niveles_escalados[n]
        ax.hlines(y=y, xmin=0, xmax=1, color='gray', linestyle='--')
        ax.text(1.2, y, f'n = {n}', verticalalignment='center', fontsize=10, color='darkblue')

    if n1 < n2:
        y1 = niveles_escalados[n1]
        y2 = niveles_escalados[n2]
        lambda_nm = seleccionar_modelo(modelo, n1, n2, Z)
        tipo = clasificar_radiacion(lambda_nm)
        ax.annotate('', xy=(0.5, y1), xytext=(0.5, y2),
                    arrowprops=dict(arrowstyle='->', color='blue', lw=2))
        ax.text(0.52, (y1 + y2)/2, f"{lambda_nm:.1f} nm\n({tipo})", fontsize=12, color='blue', ha='center')
        ax.set_title(f"Modelo: {modelo} | Z = {Z} | Transición: n={n2} → n={n1} | {lambda_nm:.1f} nm ({tipo})", fontsize=14, fontweight='bold')
    else:
        lambda_nm = None
        ax.set_title("Selecciona n₂ > n₁ para ver una transición (emisión)", fontsize=14, fontweight='bold')

    ax.set_xlim(0, 1.5)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

    if lambda_nm:
        mostrar_espectro(lambda_nm)

# Widgets interactivos
n1_slider = widgets.IntSlider(value=1, min=1, max=6, step=1, description='n₁ (final):')
n2_slider = widgets.IntSlider(value=2, min=2, max=7, step=1, description='n₂ (inicial):')
Z_dropdown = widgets.Dropdown(options=[('Hidrógeno (Z=1)', 1), ('Helio ionizado (Z=2)', 2)],
                              value=1, description='Elemento:')
modelo_dropdown = widgets.Dropdown(options=['Bohr', 'Sommerfeld', 'Schrödinger', 'Rutherford'],
                                   value='Bohr', description='Modelo Atómico:')

ui = widgets.VBox([modelo_dropdown, n1_slider, n2_slider, Z_dropdown])
out = widgets.interactive_output(graficar_transicion, {'modelo': modelo_dropdown, 'n1': n1_slider, 'n2': n2_slider, 'Z': Z_dropdown})

display(ui, out)





Output()